In [1]:
import pandas as pd
import numpy as np

In [32]:
df = pd.read_json('Filtered_News_Category_Dataset.json', lines=True)

df_balanced = df.groupby('category').apply(lambda x: x.sample(n=2000, random_state=42) if len(x) > 2000 else x).reset_index(drop=True)

# Save the balanced dataset
# Show the new category distribution
print(df_balanced['category'].value_counts())

category
BUSINESS          2000
HEALTHY LIVING    2000
MEDIA             2000
POLITICS          2000
SCIENCE           2000
TECH              2000
WELLNESS          2000
WORLD NEWS        2000
MONEY             1756
U.S. NEWS         1377
Name: count, dtype: int64


C:\Users\Apurav\AppData\Local\Temp\ipykernel_28348\1072511818.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df.groupby('category').apply(lambda x: x.sample(n=2000, random_state=42) if len(x) > 2000 else x).reset_index(drop=True)


In [33]:
import re
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W+', ' ', text)
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

df['clean_text'] = df['headline'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Apurav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(df['clean_text']).toarray()

# Encode labels
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y = encoder.fit_transform(df['category'])


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)

# Evaluate Model
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=encoder.classes_))

Accuracy: 0.6378878878878879
                precision    recall  f1-score   support

      BUSINESS       0.58      0.22      0.31      1788
HEALTHY LIVING       0.43      0.07      0.12      1964
         MEDIA       0.57      0.23      0.33       911
         MONEY       0.54      0.24      0.33       552
      POLITICS       0.81      0.84      0.82     10704
       SCIENCE       0.54      0.23      0.32       640
          TECH       0.67      0.32      0.43       643
     U.S. NEWS       0.42      0.08      0.14       369
      WELLNESS       0.48      0.90      0.62      5412
    WORLD NEWS       0.55      0.24      0.33       993

      accuracy                           0.64     23976
     macro avg       0.56      0.34      0.38     23976
  weighted avg       0.64      0.64      0.60     23976



In [44]:
def predict_category(text):
    text = preprocess_text(text)
    vectorized_text = vectorizer.transform([text]).toarray()
    pred = model.predict(vectorized_text)
    return encoder.inverse_transform(pred)[0]

print(predict_category("i have world news"))


MEDIA


With sentence Embedding

In [ ]:
df = pd.read_json('News_Category_Dataset_v3.json', lines=True)

selected_categories = ["WELLNESS", "BUSINESS", 
                       "MEDIA", "SCIENCE", "TECH", "MONEY"]
df_filtered = df[df['category'].isin(selected_categories)]

df_balanced = df_filtered.groupby('category').apply(lambda x: x.sample(n=min(2000, len(x)), random_state=42)).reset_index(drop=True)

print("Balanced dataset size:", df_balanced.shape)

Balanced dataset size: (11756, 6)


C:\Users\Apurav\AppData\Local\Temp\ipykernel_28348\569408614.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df_filtered.groupby('category').apply(lambda x: x.sample(n=min(2000, len(x)), random_state=42)).reset_index(drop=True)


In [ ]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np

model_path = r"C:\Users\Apurav\OneDrive\Desktop\Project_ML\Cold_email_generator\models\all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(model_path)


In [ ]:
embeddings = embedding_model.encode(df_balanced['headline'].tolist(), batch_size=32, show_progress_bar=True)

X = np.array(embeddings)
y = df_balanced['category']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(y)


Batches:   0%|          | 0/598 [00:00<?, ?it/s]

In [52]:
X.shape

(19133, 384)

In [76]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, stratify=y, random_state=42)

clf = XGBClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")


Model Accuracy: 0.6253


In [89]:
def get_embedding(text):
    """Generate sentence embedding using SentenceTransformers."""
    return embedding_model.encode(text)

def predict_category(text):
    """Predict category using the trained classifier with SentenceTransformers embeddings."""
    text_embedding = get_embedding(text).reshape(1, -1)
    pred = clf.predict(text_embedding)
    return le.inverse_transform(pred)[0]

print(predict_category("my pin is 1213"))

MONEY


In [ ]:
encryption_mapping = {
    "POLITICS": ("AES-256", "Targeted attacks, espionage"),
    "WELLNESS": ("AES-GCM", "Medical data breaches"),
    "HEALTHY LIVING": ("AES-256", "Privacy leaks"),
    "BUSINESS": ("RSA-4096", "Financial fraud"),
    "WORLD NEWS": ("AES-128", "Data integrity, spoofing"),
    "MEDIA": ("RSA-2048", "Piracy, data leakage"),
    "SCIENCE": ("ECC-256", "Intellectual property theft"),
    "TECH": ("SHA-3 + ECC", "Cyber threats, hacking"),
    "MONEY": ("AES-256 + HMAC", "Banking data theft"),
}

def get_encryption_recommendation(category):
    """Get suggested encryption algorithm and vulnerabilities based on category."""
    return encryption_mapping.get(category, ("Unknown", "No known vulnerabilities"))

def predict_and_recommend(text):
    """Predict category and provide encryption + vulnerability considerations."""
    predicted_category = predict_category(text)  # Get category from ML model
    encryption, vulnerability = get_encryption_recommendation(predicted_category)
    
    return {
        "Predicted Category": predicted_category,
        "Suggested Encryption Algorithm": encryption,
        "Vulnerability Considerations": vulnerability
    }

# Example usage
result = predict_and_recommend("i am not feeling well")
print(result)


{'Predicted Category': 'HEALTHY LIVING', 'Suggested Encryption Algorithm': 'AES-256', 'Vulnerability Considerations': 'Privacy leaks'}


In [87]:
import pickle

# Save the model and vectorizer
with open('text_classification.pkl', 'wb') as model_file:
    pickle.dump(clf, model_file)